# Spesifikasi

- V<sub>in</sub> = 48V
- V<sub>out</sub> = 12V
- P<sub>o</sub> = 100W
- &#916;V<sub>o</sub> &#8804; 1%
- &#916;I<sub>L</sub> &#8804; 10%
- f<sub>sw</sub> = 20e3
- Mosfet: IRFB4310PbF, Infineon

# Plecs

In [46]:
import xmlrpc.client as xml
import os
import numpy as np
import itertools
import csv

In [47]:
model = 'buck'
file_type = '.plecs'
V_in = 48
V_out = 12
f_sw = 20e3

In [48]:
plecs = xml.Server("http://localhost:1080/RPC2").plecs

In [49]:
# plecs.load(r"D:\Krishna\ai-power-converter\buck.plecs")
plecs.load(r"E:\ai-power-converter\buck.plecs")

0

In [50]:
plecs.get(model+'/L1')

{'CommentStatus': 'Active',
 'L': '0.002',
 'Name': 'L1',
 'Type': 'Inductor',
 'i_init': '0'}

In [51]:
plecs.get(model+'/C')

{'C': '0.0002',
 'CommentStatus': 'Active',
 'Name': 'C',
 'Type': 'Capacitor',
 'v_init': '0'}

In [52]:
plecs.get(model+'/Symmetrical PWM1')

{'CommentStatus': 'Active',
 'Name': 'Symmetrical PWM1',
 'Type': 'Subsystem',
 'carrier_limits': '[0 1]',
 'carrier_phaseshift': '0',
 'fc': '200000',
 'output_values': '[-1 1]',
 'sampling': 'Natural (carrier starts at center)'}

In [53]:
plecs.get(model+'/RL')

{'CommentStatus': 'Active', 'Name': 'RL', 'R': '0.443394', 'Type': 'Resistor'}

In [54]:
RL = plecs.get(model+'/RL', 'R')

In [55]:
plecs.get(model+'/ESR')

{'CommentStatus': 'Active',
 'Name': 'ESR',
 'R': '0.0005570423008216338',
 'Type': 'Resistor'}

In [56]:
plecs.get(model+'/ESL')

{'CommentStatus': 'Active',
 'L': '2.1945e-07',
 'Name': 'ESL',
 'Type': 'Inductor',
 'i_init': '0'}

In [57]:
plecs.get(model+'/Deadtime')

{'CommentStatus': 'Active',
 'Configuration': 'on',
 'Name': 'Deadtime',
 'Type': 'ConfigurableSubsystem',
 'td': '1.78e-07'}

# Average Current

In [64]:
V_in = 48
V_out = 12
R_on = 5.6
t_rise = 110e-9
t_fall = 78e-9
diode_voltage = 0.5

L_range = (30e-6, 2000e-6)  # Range for L
C_range = (30e-6, 200e-6)   # Range for C
# t_dt_range = (0.178e-6, 1e-6) # Range for dead time
fsw_range = (20e3, 200e3)    # Range for fsw
num_values = 3           # Number of values for each parameter

# Borders for R_L
border_values = [
    (144e-6, 0.079576),
    (509e-6, 0.179228),
    (1003e-6, 0.253239),
    (2519e-6, 0.443394)
]

# Constants for ESL calculation
K_esl = 4.389e-11  # Adjust as needed
tan_delta = 0.14

# Create arrays of discrete L and C values within the specified ranges with increments of 1e-6
L_values = np.linspace(L_range[0], L_range[1], num=num_values)
C_values = np.linspace(C_range[0], C_range[1], num=num_values)
# t_dt_values = np.linspace(t_dt_range[0], t_dt_range[1], num=num_values)


# Round the values to the desired number of decimal places
L_values = np.around(L_values, decimals=6)
C_values = np.around(C_values, decimals=6)
# t_dt_values = np.around(t_dt_values, decimals=9)

fsw_values = np.linspace(fsw_range[0], fsw_range[1], num=num_values, dtype=int)

# Initialize arrays to store results
average_currents = []
delta_currents = []
delta_currents_percentage = []

# Open a CSV file for writing
csv_file_path = 'average_current_3_3.csv'

# Define the header for the CSV file
csv_header = ['No','L', 'C','fsw','average_current']

# Initialize a list to store the data for each simulation
csv_data = []

# Print the chosen values
print("Chosen L values:", L_values)
print("Chosen C values:", C_values)
# print("Chosen dead time values:", t_dt_values)
print("Chosen fsw values:", fsw_values)

# Generate all combinations of L, C, and fsw
combinations = list(itertools.product(L_values, C_values, fsw_values))
print(combinations)
# Display the chosen values and simulate
print("\nChosen values and simulation results:")

Chosen L values: [3.000e-05 1.015e-03 2.000e-03]
Chosen C values: [3.00e-05 1.15e-04 2.00e-04]
Chosen fsw values: [ 20000 110000 200000]
[(3e-05, 3e-05, 20000), (3e-05, 3e-05, 110000), (3e-05, 3e-05, 200000), (3e-05, 0.000115, 20000), (3e-05, 0.000115, 110000), (3e-05, 0.000115, 200000), (3e-05, 0.0002, 20000), (3e-05, 0.0002, 110000), (3e-05, 0.0002, 200000), (0.001015, 3e-05, 20000), (0.001015, 3e-05, 110000), (0.001015, 3e-05, 200000), (0.001015, 0.000115, 20000), (0.001015, 0.000115, 110000), (0.001015, 0.000115, 200000), (0.001015, 0.0002, 20000), (0.001015, 0.0002, 110000), (0.001015, 0.0002, 200000), (0.002, 3e-05, 20000), (0.002, 3e-05, 110000), (0.002, 3e-05, 200000), (0.002, 0.000115, 20000), (0.002, 0.000115, 110000), (0.002, 0.000115, 200000), (0.002, 0.0002, 20000), (0.002, 0.0002, 110000), (0.002, 0.0002, 200000)]

Chosen values and simulation results:


# Functions

## Write to CSV

In [14]:
def write_to_csv(file_path, header, data):
    with open(file_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(header)
        csv_writer.writerows(data)

## Ripples

### Current
- Average
- &Delta;I<sub>L</sub>%

In [15]:
def calculate_current(times, current, start_time, end_time):
    # Convert 'times' to a NumPy array
    times = np.array(times)

    # Find the indices corresponding to the time range
    start_index = np.argmax(times >= start_time)
    end_index = np.argmax(times >= end_time)

    # Extract the current values in the specified time range
    current_range = current[start_index:end_index]

    # Calculate the average current value
    average_current = np.mean(current_range)

    # Calculate the difference between the highest and lowest current values
    delta_current = np.max(current_range) - np.min(current_range)

    # print(np.max(current_range))
    # print(np.min(current_range))
    delta_current_percentage = delta_current/average_current *100

    return average_current, delta_current, delta_current_percentage

### &Delta;V<sub>O</sub>%

In [16]:
def calculate_voltage(times, voltage, start_time, end_time):
    # Convert 'times' to a NumPy array
    times = np.array(times)

    # Find the indices corresponding to the time range
    start_index = np.argmax(times >= start_time)
    end_index = np.argmax(times >= end_time)

    # Extract the voltage values in the specified time range
    voltage_range = voltage[start_index:end_index]

    # Calculate the average voltage value
    average_voltage = np.mean(voltage_range)

    # Calculate the difference between the highest and lowest voltage values
    delta_voltage = np.max(voltage_range) - np.min(voltage_range)

    print('Voltage:')
    print("\nmax:", np.max(voltage_range))
    print("\nmin:", np.min(voltage_range))
    print("\ndelta:",delta_voltage)
    print("\naverage:",average_voltage)

    delta_voltage_percentage = delta_voltage/average_voltage * 100

    return average_voltage, delta_voltage, delta_voltage_percentage


## Switching Losses
- Conduction Loss
    - R<sub>DS(on)</sub> = R<sub>ON-H</sub> = R<sub>ON-L</sub> = 5.6k&Omega; 
    - High Side
        - P<sub>ON-H</sub> = I<sub>OUT</sub> / 2 &times; R<sub>ON-H</sub> &times; V<sub>OUT</sub> / V<sub>IN</sub> [W]
    - Low Side
        - P<sub>ON-L</sub> = I<sub>OUT</sub> / 2 &times; R<sub>ON-L</sub> &times; (1 - V<sub>OUT</sub> / V<sub>IN</sub>) [W]
- Switching Losses
    - t<sub>rise</sub> = 110 ns  
    - t<sub>fall</sub> = 78 ns
    - High Side
        - P<sub>SW-H</sub> = 1/2 &times; V<sub>IN</sub> &times; I<sub>OUT</sub> &times; (t<sub>rise-H</sub> + t<sub>fall-H</sub>) &times; f<sub>SW</sub>
    - Low Side
        - P<sub>SW-L</sub> = 1/2 &times; V<sub>D</sub> &times; I<sub>OUT</sub> &times; (t<sub>rise-L</sub> + t<sub>fall-L</sub>) &times; f<sub>SW</sub>



<strong>Switching Losses<strong> = P<sub>ON-H</sub> + P<sub>ON-L</sub> + P<sub>SW-H</sub> + P<sub>SW-L</sub>

In [17]:
def calculate_switching_losses(average_current, V_out, V_in, R_on, t_rise, t_fall, f_sw):
    # Calculate power losses during ON state (High and Low)
    P_ON_H = average_current / 2 * R_on * V_out / V_in
    P_ON_L = average_current / 2 * R_on * V_out / V_in

    # Calculate power losses during switching (High and Low)
    P_SW_H = 0.5 * V_in * average_current * (t_rise + t_fall) * f_sw
    P_SW_L = 0.5 * V_in * average_current * (t_rise + t_fall) * f_sw

    # High Side
    Pl_s1 = P_ON_H + P_SW_H

    #Low Side
    Pl_s2 = P_ON_L + P_SW_L

    return Pl_s1, Pl_s2


## Capacitor Loss
- P<sub>CAP(ESR)</sub> = I<sub>CAP(RMS)</sub><sup>2</sup>  &#x00D7; ESR [W]
- ESR = tan(&#x03B4;) / 2&pi;f<sub>SW</sub>C)
    - Dengan 25 V Nippon KZE series: tan &#x03B4; = 0.14
- ESL = 1 / (4π² SRF² C)
    - Assume SRF = 100kHz
- Input Capacitor
    - I<sub>CIN(RMS)</sub> = I<sub>OUT</sub> &times; &radic;(V<sub>IN</sub> - V<sub>OUT</sub>) &times; V<sub>OUT</sub> / V<sub>IN</sub> [A]
- Output Capacitor
    - I<sub>COUT(RMS)</sub> = &Delta;I<sub>L</sub> / (2&radic;3) [A]
    - &Delta;I<sub>L</sub> = (V<sub>IN</sub> - V<sub>OUT</sub>) / (f<sub>SW</sub> &times; L) &times; V<sub>OUT</sub> / V<sub>IN</sub> [A]

<strong>Capacitor Losses<strong> = P<sub>cap_in</sub> + P<sub>cap_out</sub>

In [18]:
def calculate_capacitor_losses(average_current, V_in, V_out, f_sw, L, ESR):
    I_cin = average_current * np.sqrt(V_in - V_out) * V_out / V_in
    I_L = ((V_in - V_out) / (f_sw * L)) * (V_out / V_in)

    I_cout = I_L / (2 * np.sqrt(3))
    P_cap_in = I_cin**2 * ESR
    P_cap_out = I_cout**2 * ESR

    Pl_C = P_cap_in + P_cap_out

    return Pl_C


## Inductor Loss
- Inductor Copper Loss
    - P<sub>Copper, L</sub> = I<sub>RMS</sub><sup>2</sup> &middot; R<sub>L</sub>
<br>
    - I<sub>RMS</sub> = &radic;(I<sub>AVG</sub><sup>2</sup> + <sup>1</sup>&frasl;<sub>12</sub> &Delta;I<sub>L</sub><sup>2</sup>)

In [19]:
def calculate_inductor_copper_loss(average_current, delta_current, R_L):
    I_rms = np.sqrt(average_current**2 + 1/12 * delta_current**2)
    Pl_L_Cu = I_rms**2 * R_L

    return Pl_L_Cu

## Dead time losses

- t<sub>d(on)</sub> = 26 nS
- t<sub>d(off)</sub> = 68 nS
- t<sub>r</sub> = 110 nS
- t<sub>f</sub> = 78 nS
- Safety factor = 20%
- V<sub>D</sub> = 0.5V

<strong>t<sub>dt</sub><strong> = (t<sub>r</sub> + t<sub>d(off)</sub>) = 178 nS

<strong>t<sub>dt</sub><strong> = t<sub>Dr</sub> = t<sub>Df</sub>   

<strong>P<sub>D</sub></strong> = V<sub>D</sub> × I<sub>out</sub> × (t<sub>Dr</sub> + t<sub>Df</sub>) × f<sub>sw</sub>


In [20]:
def calculate_dead_time_loss(average_current, f_sw, t_dt):
    P_D = diode_voltage * average_current * 2 * t_dt * f_sw

    return P_D

# Single

In [31]:
val_test =[3e-05,3e-05,20000]

L = val_test[0]
C = val_test[1]
fsw = val_test[2]
# t_dt = 1e-6

R_L = next((rl for border, rl in border_values if L <= border), 0.0)

# Calculate ESR and ESL based on the updated C
ESR = tan_delta / (2 * np.pi * fsw * C)
ESL = K_esl / C

# Set Plecs parameters and simulate
plecs.set(model+'/L1', 'L', str(L))
plecs.set(model+'/C', 'C', str(C))
plecs.set(model+'/Symmetrical PWM1', 'fc', str(fsw))
plecs.set(model+'/RL', 'R', str(R_L))
plecs.set(model+'/ESR', 'R', str(ESR))
plecs.set(model+'/ESL', 'L', str(ESL))
# plecs.set(model+'/Deadtime', 'td', str(t_dt))

times = plecs.simulate(model)['Time']
current = plecs.simulate(model)['Values'][0]
voltage = plecs.simulate(model)['Values'][1]

# Calculate average and delta current using the function
average_current, delta_current, delta_current_percentage = calculate_current(times, current, start_time=0.004, end_time=0.005)
# average_voltage, delta_voltage, delta_voltage_percentage = calculate_voltage(times, voltage, start_time=0.004, end_time=0.005)
# Pl_s1, Pl_s2 = calculate_switching_losses(average_current, V_out, V_in, R_on, t_rise, t_fall, fsw)
# Pl_C = calculate_capacitor_losses(average_current, V_in, V_out, fsw, L, ESR)
# Pl_L_Cu = calculate_inductor_copper_loss(average_current, delta_current, R_L)
# P_D = calculate_dead_time_loss(average_current, fsw, t_dt)


print(f"Average Current between 0.004 and 0.005 seconds: {average_current} A")
print(f"Difference between highest and lowest current values between 0.004 and 0.005 seconds: {delta_current} A")
print(f"Difference between highest and lowest current values between 0.004 and 0.005 seconds: {delta_current_percentage} %")
# print(f"Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: {delta_voltage} V")
# print(f"Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: {delta_voltage_percentage} %")
# print(f"High Side Switching Losses: {Pl_s1} W")
# print(f"Low Side Switching  Losses: {Pl_s2} W")
# print(f"Capacitor Losses: {Pl_C} W")
# print(f"Inductor Copper Losses: {Pl_L_Cu} W")
# print(f"Dead time Losses: {P_D} W")
# print(f"Total Losses: {Pl_s1 + Pl_s2 + Pl_C + Pl_L_Cu + P_D} W")

Average Current between 0.004 and 0.005 seconds: 3.2556026550071735 A
Difference between highest and lowest current values between 0.004 and 0.005 seconds: 14.944351228442894 A
Difference between highest and lowest current values between 0.004 and 0.005 seconds: 459.0348642657058 %


# Loop

In [65]:
# Loop through the combinations and simulate
csv_data = []
for simulation_num, (L, C, fsw) in enumerate(combinations, start=1):
    # Rules for picking R_L based on L
    R_L = next((rl for border, rl in border_values if L <= border), 0.0)

    # Calculate ESR and ESL based on the updated C
    ESR = tan_delta / (2 * np.pi * fsw * C)
    ESL = K_esl / C

    # Set Plecs parameters and simulate
    plecs.set(model+'/L1', 'L', str(L))
    plecs.set(model+'/C', 'C', str(C))
    plecs.set(model+'/Symmetrical PWM1', 'fc', str(fsw))
    plecs.set(model+'/RL', 'R', str(R_L))
    plecs.set(model+'/ESR', 'R', str(ESR))
    plecs.set(model+'/ESL', 'L', str(ESL))
    # plecs.set(model+'/Deadtime', 'td', str(t_dt))

    times = plecs.simulate(model)['Time']
    current = plecs.simulate(model)['Values'][0]
    voltage = plecs.simulate(model)['Values'][1]

    # Calculate average and delta current using the function
    average_current, delta_current, delta_current_percentage = calculate_current(times, current, start_time=0.004, end_time=0.005)
    # average_voltage, delta_voltage, delta_voltage_percentage = calculate_voltage(times, voltage, start_time=0.004, end_time=0.005)
    # Pl_s1, Pl_s2 = calculate_switching_losses(average_current, V_out, V_in, R_on, t_rise, t_fall, fsw)
    # Pl_C = calculate_capacitor_losses(average_current, V_in, V_out, fsw, L, ESR)
    # Pl_L_Cu = calculate_inductor_copper_loss(average_current, delta_current, R_L)
    # P_D = calculate_dead_time_loss(average_current, fsw, t_dt)

    # Append the data for this simulation to the list
    csv_data.append([simulation_num,L, C, fsw, average_current])

    print(f'\nSimulation {simulation_num} for L={L:.6f}, C={C:.6f}, fsw={fsw:.2f}, RL={R_L:.6f}, ESR={ESR:.6f}, ESL={ESL:.6e}')
    print(f"\nSimulation {simulation_num} results:")
    print(f"Average Current between 0.004 and 0.005 seconds: {average_current} A")
    # print(f"Difference between highest and lowest current values between 0.004 and 0.005 seconds: {delta_current} A")
    # print(f"Difference between highest and lowest current values between 0.004 and 0.005 seconds: {delta_current_percentage} %")
    # print(f"Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: {delta_voltage} A")
    # print(f"Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: {delta_voltage_percentage} %")
    # print(f"High Side Switching Losses: {Pl_s1} W")
    # print(f"Low Side Switching  Losses: {Pl_s2} W")
    # print(f"Capacitor Losses: {Pl_C} W")
    # print(f"Inductor Copper Losses: {Pl_L_Cu} W")
    # # print(f"Dead time Losses: {P_D} W")
    # print(f"Total Losses: {Pl_s1 + Pl_s2 + Pl_C + Pl_L_Cu} W")


Simulation 1 for L=0.000030, C=0.000030, fsw=20000.00, RL=0.079576, ESR=0.037136, ESL=1.463000e-06

Simulation 1 results:
Average Current between 0.004 and 0.005 seconds: 2.369076212400351 A

Simulation 2 for L=0.000030, C=0.000030, fsw=110000.00, RL=0.079576, ESR=0.006752, ESL=1.463000e-06

Simulation 2 results:
Average Current between 0.004 and 0.005 seconds: 2.2609969793405247 A

Simulation 3 for L=0.000030, C=0.000030, fsw=200000.00, RL=0.079576, ESR=0.003714, ESL=1.463000e-06

Simulation 3 results:
Average Current between 0.004 and 0.005 seconds: 2.082148933928239 A

Simulation 4 for L=0.000030, C=0.000115, fsw=20000.00, RL=0.079576, ESR=0.009688, ESL=3.816522e-07

Simulation 4 results:
Average Current between 0.004 and 0.005 seconds: 2.1876638438148737 A

Simulation 5 for L=0.000030, C=0.000115, fsw=110000.00, RL=0.079576, ESR=0.001761, ESL=3.816522e-07

Simulation 5 results:
Average Current between 0.004 and 0.005 seconds: 2.0157392163492425 A

Simulation 6 for L=0.000030, C=0.

In [66]:
# Use the function to write data to the CSV file
write_to_csv(csv_file_path, csv_header, csv_data)

# All

In [59]:
V_in = 48
V_out = 12
R_on = 5.6
t_rise = 110e-9
t_fall = 78e-9
diode_voltage = 0.5

L_range = (30e-6, 2000e-6)  # Range for L
C_range = (30e-6, 200e-6)   # Range for C
# t_dt_range = (0.178e-6, 1e-6) # Range for dead time
fsw_range = (20e3, 200e3)    # Range for fsw
num_values = 3           # Number of values for each parameter

# Borders for R_L
border_values = [
    (144e-6, 0.079576),
    (509e-6, 0.179228),
    (1003e-6, 0.253239),
    (2519e-6, 0.443394)
]

# Constants for ESL calculation
K_esl = 4.389e-11  # Adjust as needed
tan_delta = 0.14

# Create arrays of discrete L and C values within the specified ranges with increments of 1e-6
L_values = np.linspace(L_range[0], L_range[1], num=num_values)
C_values = np.linspace(C_range[0], C_range[1], num=num_values)
# t_dt_values = np.linspace(t_dt_range[0], t_dt_range[1], num=num_values)


# Round the values to the desired number of decimal places
L_values = np.around(L_values, decimals=6)
C_values = np.around(C_values, decimals=6)
# t_dt_values = np.around(t_dt_values, decimals=9)

fsw_values = np.linspace(fsw_range[0], fsw_range[1], num=num_values, dtype=int)

# Initialize arrays to store results
average_currents = []
delta_currents = []
delta_currents_percentage = []

# Open a CSV file for writing
csv_file_path = 'all_3_3.csv'

# Define the header for the CSV file
csv_header = ['No','L', 'C', 'delta_current', 'delta_voltage', 'Pl_s1', 'Pl_s2' 'Pl_C', 'Pl_L_Cu']

# Initialize a list to store the data for each simulation
csv_data = []

# Print the chosen values
print("Chosen L values:", L_values)
print("Chosen C values:", C_values)
# print("Chosen dead time values:", t_dt_values)
print("Chosen fsw values:", fsw_values)

# Generate all combinations of L, C, and fsw
combinations = list(itertools.product(L_values, C_values, fsw_values))
print(combinations)
# Display the chosen values and simulate
print("\nChosen values and simulation results:")

Chosen L values: [3.000e-05 1.015e-03 2.000e-03]
Chosen C values: [3.00e-05 1.15e-04 2.00e-04]
Chosen fsw values: [ 20000 110000 200000]
[(3e-05, 3e-05, 20000), (3e-05, 3e-05, 110000), (3e-05, 3e-05, 200000), (3e-05, 0.000115, 20000), (3e-05, 0.000115, 110000), (3e-05, 0.000115, 200000), (3e-05, 0.0002, 20000), (3e-05, 0.0002, 110000), (3e-05, 0.0002, 200000), (0.001015, 3e-05, 20000), (0.001015, 3e-05, 110000), (0.001015, 3e-05, 200000), (0.001015, 0.000115, 20000), (0.001015, 0.000115, 110000), (0.001015, 0.000115, 200000), (0.001015, 0.0002, 20000), (0.001015, 0.0002, 110000), (0.001015, 0.0002, 200000), (0.002, 3e-05, 20000), (0.002, 3e-05, 110000), (0.002, 3e-05, 200000), (0.002, 0.000115, 20000), (0.002, 0.000115, 110000), (0.002, 0.000115, 200000), (0.002, 0.0002, 20000), (0.002, 0.0002, 110000), (0.002, 0.0002, 200000)]

Chosen values and simulation results:


In [60]:
# Loop through the combinations and simulate
csv_data = []
for simulation_num, (L, C, fsw) in enumerate(combinations, start=1):
    # Rules for picking R_L based on L
    R_L = next((rl for border, rl in border_values if L <= border), 0.0)

    # Calculate ESR and ESL based on the updated C
    ESR = tan_delta / (2 * np.pi * fsw * C)
    ESL = K_esl / C

    # Set Plecs parameters and simulate
    plecs.set(model+'/L1', 'L', str(L))
    plecs.set(model+'/C', 'C', str(C))
    plecs.set(model+'/Symmetrical PWM1', 'fc', str(fsw))
    plecs.set(model+'/RL', 'R', str(R_L))
    plecs.set(model+'/ESR', 'R', str(ESR))
    plecs.set(model+'/ESL', 'L', str(ESL))
    # plecs.set(model+'/Deadtime', 'td', str(t_dt))

    times = plecs.simulate(model)['Time']
    current = plecs.simulate(model)['Values'][0]
    voltage = plecs.simulate(model)['Values'][1]

    # Calculate average and delta current using the function
    average_current, delta_current, delta_current_percentage = calculate_current(times, current, start_time=0.004, end_time=0.005)
    average_voltage, delta_voltage, delta_voltage_percentage = calculate_voltage(times, voltage, start_time=0.004, end_time=0.005)
    Pl_s1, Pl_s2 = calculate_switching_losses(average_current, V_out, V_in, R_on, t_rise, t_fall, fsw)
    Pl_C = calculate_capacitor_losses(average_current, V_in, V_out, fsw, L, ESR)
    Pl_L_Cu = calculate_inductor_copper_loss(average_current, delta_current, R_L)
    # P_D = calculate_dead_time_loss(average_current, fsw, t_dt)

    # Append the data for this simulation to the list
    csv_data.append([simulation_num,L, C, fsw, delta_current_percentage, delta_voltage_percentage, Pl_s1, Pl_s2, Pl_C, Pl_L_Cu])

    print(f'\nSimulation {simulation_num} for L={L:.6f}, C={C:.6f}, fsw={fsw:.2f}, RL={R_L:.6f}, ESR={ESR:.6f}, ESL={ESL:.6e}')
    print(f"\nSimulation {simulation_num} results:")
    print(f"Average Current between 0.004 and 0.005 seconds: {average_current} A")
    print(f"Difference between highest and lowest current values between 0.004 and 0.005 seconds: {delta_current} A")
    print(f"Difference between highest and lowest current values between 0.004 and 0.005 seconds: {delta_current_percentage} %")
    print(f"Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: {delta_voltage} A")
    print(f"Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: {delta_voltage_percentage} %")
    print(f"High Side Switching Losses: {Pl_s1} W")
    print(f"Low Side Switching  Losses: {Pl_s2} W")
    print(f"Capacitor Losses: {Pl_C} W")
    print(f"Inductor Copper Losses: {Pl_L_Cu} W")
    # print(f"Dead time Losses: {P_D} W")
    print(f"Total Losses: {Pl_s1 + Pl_s2 + Pl_C + Pl_L_Cu} W")

Voltage:

max: 12.679319769676034

min: 10.055395242914386

delta: 2.6239245267616482

average: 11.754836343978347

Simulation 1 for L=0.000030, C=0.000030, fsw=20000.00, RL=0.079576, ESR=0.037136, ESL=1.463000e-06

Simulation 1 results:
Average Current between 0.004 and 0.005 seconds: 2.369076212400351 A
Difference between highest and lowest current values between 0.004 and 0.005 seconds: 14.942629474599412 A
Difference between highest and lowest current values between 0.004 and 0.005 seconds: 630.7365460167921 %
Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: 2.6239245267616482 A
Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: 22.322084714567776 %
High Side Switching Losses: 1.8721387860872531 W
Low Side Switching  Losses: 1.8721387860872531 W
Capacitor Losses: 1.1652647096399509 W
Inductor Copper Losses: 1.927280592538472 W
Total Losses: 6.836822874352929 W
Voltage:

max: 12.55893532660962

min: 10.357033046

In [61]:
# Use the function to write data to the CSV file
write_to_csv(csv_file_path, csv_header, csv_data)